In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## 1. Análisis de Procedimientos a Nivel País

En este análisis se quieren responder las siguiente preguntas:

1. ¿Cómo es el ranking de los hospitales para cada uno de los procedimientos realizads?
2. ¿Quién es el Hospital que realiza más de X procedimientos?
3. ¿El procedimiento X es realizado en todo el país? ¿En qué partes en específico se realiza?

Para responder esta pregunta se utilizará la base de datos de Egresos de Hospitales adscritos
a GRD entre 2019 y 2021.


In [3]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";")
df_procesada["ANIO_EGRESO"] = df_procesada["ANIO_EGRESO"].astype("Int32")

In [4]:
display(df_procesada.head(10))

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA
0,118100,1314867.0,MUJER,1988-06-19,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,1.0,146102.0,0.5744,2.0,1.0,NaN,5.0,2019,3.0,2019-3
1,118100,1354418.0,MUJER,1988-03-27,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,NaN,134161.0,0.2951,1.0,1.0,NaN,4.0,2019,3.0,2019-3
2,118100,239861.0,MUJER,2017-06-17,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,64182.0,0.6736,2.0,2.0,NaN,6.0,2019,3.0,2019-3
3,118100,330326.0,HOMBRE,2017-12-04,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,NaN,64172.0,0.5475,2.0,2.0,NaN,2.0,2019,3.0,2019-3
4,118100,1369293.0,MUJER,1995-12-06,OTRO,CONCEPCION,FLORIDA,CHILE,FONASA INSTITUCIONAL - (MAI) C,CONCEPCIÓN,...,NaN,134141.0,0.3107,1.0,1.0,NaN,6.0,2019,3.0,2019-3
5,118100,469385.0,MUJER,1942-06-29,OTRO,CONCEPCION,CHIGUAYANTE,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,114103.0,1.1799,3.0,3.0,NaN,47.0,2019,3.0,2019-3
6,118100,1146635.0,MUJER,1951-07-14,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,44163.0,1.0328,3.0,3.0,NaN,43.0,2019,3.0,2019-3
7,118100,93557.0,HOMBRE,1938-03-04,OTRO,CONCEPCION,LOTA,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,1.0,11101.0,1.8731,1.0,1.0,NaN,41.0,2019,3.0,2019-3
8,118100,674943.0,MUJER,1962-08-21,OTRO,CONCEPCION,SANTA JUANA,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,1.0,61402.0,1.3078,2.0,2.0,HOSPITAL CLORINDA AVELLO (SANTA JUANA),33.0,2019,3.0,2019-3
9,118100,1079955.0,HOMBRE,1944-12-07,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,14133.0,1.6687,3.0,3.0,NaN,31.0,2019,3.0,2019-3


Los procedimientos en esta base de datos se encuentran en las columnas "PROCEDIMIENTOSX", donde X
va desde 1 hasta 30. Para analizar esta base de forma más expedita, se transformará de formato
ancho a formato largo, utilizando como identificador a las columnas ["COD_HOSPITAL", "ANIO_EGRESO",
"CIP_ENCRIPTADO", "DIAGNOSTICO1"]. Cabe destacar que la columna "CIP_ENCRIPTADO" hace referencia al
RUT del paciente.


In [5]:
procedimientos_formato_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "CIP_ENCRIPTADO", "DIAGNOSTICO1"],
    value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)],
    value_name="procedimiento",
)

In [6]:
conteo_y_ranking_procedimientos = build_features.contar_procedimientos_y_rankear(
    procedimientos_formato_long
)

In [7]:
display(conteo_y_ranking_procedimientos.head(20))

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo,ranking,diferencia_proximo_hospital,total_procedimientos
150206,2021,108100,99.99,2,1,0,11
165765,2021,113180,99.99,2,2,1,11
140654,2021,103100,99.99,1,3,0,11
143682,2021,105100,99.99,1,4,0,11
155556,2021,110120,99.99,1,5,0,11
167921,2021,114101,99.99,1,6,0,11
183553,2021,121109,99.99,1,7,0,11
186513,2021,122100,99.99,1,8,0,11
190822,2021,126100,99.99,1,9,-18,11
191999,2021,129106,99.98,19,1,8,119


El resultado muestra el ranking para todos los procedimientos asignados en el país en cada uno de 
los años analizados (2019 a 2021). 

A modo de ejemplo, se ilustra el ranking en el año 2021 para el procedimiento 99.99. En este caso,
se observa que los hospitales con código 108100 y 113180 están en los primeros lugares, realizando
2 procedimientos cada uno.

Ahora, si se filtra por el hospital del tórax, y sólo los procedimientos donde el hospital sea
ranking 1, entonces se obtendrán procedimientos altamente relevantes para el país.

In [12]:
mejores_procedimientos_torax = conteo_y_ranking_procedimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX and ranking == 1"
).sort_values(["ANIO_EGRESO", "conteo"],ascending=False)

In [13]:
display(mejores_procedimientos_torax)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo,ranking,diferencia_proximo_hospital,total_procedimientos
162203,2021,112103,39.61,312,1,37,1582
162205,2021,112103,39.64,294,1,135,1018
162211,2021,112103,42.23,189,1,169,304
162216,2021,112103,37.22,138,1,10,378
162221,2021,112103,88.53,129,1,11,799
...,...,...,...,...,...,...,...
31575,2019,112103,39.64,1,1,0,1
31548,2019,112103,91.61,1,1,0,3
31486,2019,112103,39.55,1,1,0,1
31476,2019,112103,38.46,1,1,0,3


Los resultados muestran que el tórax en el año 2021 fue altamente relevante en los procedimientos
39.61, 39.64, 42.23, entre otros. Además, se puede observar la diferencia que tiene el Tórax
con los siguientes establecimientos de salud y el total de procedimientos realizados en el país.